<a href="https://colab.research.google.com/github/nagbodo2/it-cert-automation-practice/blob/master/Copy_of_Capstone_Nelson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This project focuses on developing an innovative Business Intelligence Assistant using LangChain, Retrieval-Augmented Generation (RAG), and Large Language Models (LLMs) technologies. The tool combines both csv and PDF data and performs comprehensive analysis to identify key trends and patterns. Through RAG, the tool retrieves additional information from Wikipedia to generate insights and recommendations. The tool also allows data visualizations for easier interpretation. The Streamlit User Interface is added for friendly querying the data through prompts. The modules developed in the codes are Data uploading and preparation, Knowledge base creation, LLM application development, data summary, RAG system using Wikipedia corpus, Chain prompts, Memory integration, Model evaluation, Data visualization, and Streamlit User interface. Whenever applicable, we develop a function then apply the function in implementing the modules for easier readability of the code.

In [ ]:
# Installing required libraries
!pip install streamlit --quiet
!pip install langchain-community chromadb tiktoken pypdf wikipedia matplotlib --quiet

In [ ]:
### installing required packages
import streamlit as st
from IPython import get_ipython
from IPython.display import display
import streamlit as st
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.evaluation.qa import QAEvalChain
from langchain.schema import Document  # Import Document
import os
import pandas as pd
import re
import wikipedia
import matplotlib.pyplot as plt
import hashlib

In [4]:
# Define OpenAI API key once
openai_api_key = " your key"


In [5]:
### Function to upload the data/documents
def load_documents(folder_path):
    """Loads documents from the specified folder, supporting PDF and CSV files."""
    db = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith(".pdf"):
            loader = PyPDFLoader(file_path)
        elif filename.endswith(".csv"):
            loader = CSVLoader(file_path)
        else:
            continue  # Skip unsupported file types

        loaded_data = loader.load()
        db.extend(loaded_data)
        print(f"Loaded {filename}: {len(loaded_data)} documents")

    return db


In [6]:
### Function to query the sale data
def calculate_all_statistics(documents):
    """
    Calculates all possible statistics on all columns, including sum.

    Args:
        documents: List of Document objects.

    Returns:
        A dictionary containing statistics for each column.
    """
    all_data = []
    for doc in documents:
        try:
            # Data is comma-separated
            content = doc.page_content.strip()  # TO remove leading/trailing spaces
            # Split only if content is not empty
            if content:
                # Replace \t with , if present
                if "\t" in content:
                    content = content.replace("\t", ",")
                # Split content by comma
                parts = content.split(",")
                if len(parts) >= 2:  # Check for at least Product and Sales
                    data = {
                        "Product": parts[0],
                        "Sales": parts[1],
                    }
                    all_data.append(data)
                else:
                    print(f"Skipping document with invalid format: {content}")
            else:
                print("Skipping empty document")
        except ValueError as e:
            print(f"Error processing document: {e}")
            pass  # Handle errors gracefully

    df = pd.DataFrame(all_data)
    # print(df)  # Print the DataFrame to check its structure

    # Convert 'Sales' column to numeric, handling errors
    df["Sales"] = pd.to_numeric(df["Sales"], errors="coerce")

    # Calculate all statistics for all columns, including sum
    statistics = {}
    for col in df.columns:
        col_stats = df[col].describe().to_dict()
        if pd.api.types.is_numeric_dtype(df[col]):
            col_stats["sum"] = df[col].sum()
        statistics[col] = col_stats

    return statistics, df # Return the dataframe as well


In [ ]:
# Load and preprocess the data
pdf_folder_path = "/content"  # Adjust path if necessary
documents = load_documents(pdf_folder_path)
all_statistics, sales_df = calculate_all_statistics(documents)
print(f"All Statistics:\n{all_statistics}")

In [9]:
# Create embeddings
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
persist_directory = "chroma_db"

<ipython-input-9-4e170f4acab7>:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)


In [10]:
# --- Vector Database Management ---
statistics_document = [Document(page_content=f"Sales Statistics:\n{all_statistics}",
                                 metadata={"source": "sales_statistics"})]


In [11]:
# Fetch Wikipedia data with error handling, redirects, and rate limiting
import time
def fetch_wikipedia_data(query, num_results=3, sleep_time=1):
    """
    Fetches relevant Wikipedia pages based on the query.
    Handles exceptions, resolves redirects, and adds a delay between requests.
    """
    search_results = wikipedia.search(query, results=num_results)
    wikipedia_data = []
    for page_title in search_results:
        try:
            page = wikipedia.page(page_title, auto_suggest=False, redirect=True)
            wikipedia_data.append(page.content)
            time.sleep(sleep_time)  # Add a delay between requests
        except wikipedia.exceptions.DisambiguationError as e:
            print(f"Disambiguation error for {page_title}: {e.options}")
        except wikipedia.exceptions.PageError:
            print(f"Page not found for {page_title}. Trying suggestions...")
            try:
                page = wikipedia.page(page_title, auto_suggest=True, redirect=True)
                wikipedia_data.append(page.content)
                print(f"Found page using suggestion: {page.title}")
                time.sleep(sleep_time)  # Add a delay after suggestion
            except wikipedia.exceptions.PageError:
                print(f"Still could not find page for {page_title} or its suggestions.")
    return wikipedia_data

# Fetch Wikipedia data for a relevant topic (e.g., "Sales")
wikipedia_data = fetch_wikipedia_data("Sales", num_results=3)

# Handle empty Wikipedia results
if wikipedia_data:
    wikipedia_documents = [Document(page_content=content, metadata={"source": "wikipedia"})
                           for content in wikipedia_data]
else:
    wikipedia_documents = []  # Use an empty list if no Wikipedia data is found
    print("No relevant Wikipedia data found for the query.")




In [12]:
# Combine all documents
all_documents = documents + statistics_document + wikipedia_documents

In [13]:
import hashlib

def hash_document(document):
    """Hashes a Document object based on its content and metadata."""
    content_hash = hashlib.md5(document.page_content.encode()).hexdigest()
    metadata_hash = hashlib.md5(str(document.metadata).encode()).hexdigest()
    combined_hash = hashlib.md5((content_hash + metadata_hash).encode()).hexdigest()
    return combined_hash

@st.cache_resource(show_spinner=False, hash_funcs={Document: hash_document})
def create_vector_database(documents, _embeddings, persist_directory):
    """Creates a Chroma vector database from documents, embeddings, and a persist directory."""
    vectordb = Chroma.from_documents(
        documents=documents,
        embedding=_embeddings,
        persist_directory=persist_directory
    )
    return vectordb

vectordb = create_vector_database(all_documents, embeddings, persist_directory)

In [14]:
# --- Memory and Conversational Retrieval Chain ---
memory = ConversationBufferMemory(memory_key="chat_history")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0, openai_api_key=openai_api_key)

# Create chain WITHOUT caching for create_retriever_chain
def create_retriever_chain(_llm, _vectordb, memory):
    """Creates a ConversationalRetrievalChain without caching."""
    retriever_chain = ConversationalRetrievalChain.from_llm(
        llm=_llm,
        retriever=_vectordb.as_retriever(),
        memory=memory,
        return_source_documents=True
    )
    return retriever_chain

retriever_chain = create_retriever_chain(llm, vectordb, memory)


<ipython-input-14-38bbc23d2aff>:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")
<ipython-input-14-38bbc23d2aff>:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0, openai_api_key=openai_api_key)


In [15]:
# --- QAEvalChain Integration ---
# Create prediction and eval chains (no caching for eval)
prediction_chain = RetrievalQA.from_chain_type(llm=llm, retriever=vectordb.as_retriever())
eval_chain = QAEvalChain.from_llm(llm, chain_type="stuff")

def query_data(chain, question):
    result = chain({"query": question})
    answer = result["result"]
    return answer

In [16]:
# --- Data Visualization Functions ---

def plot_sales_trends(df):
    """Plots sales trends over time (assuming a 'Date' column)."""
    # For demonstration, let's assume 'Product' represents time periods
    df.groupby('Product')['Sales'].sum().plot(kind='line')
    plt.title('Sales Trends Over Time')
    plt.xlabel('Product (Time Period)')  # Replace with actual date column if available
    plt.ylabel('Sales')
    st.pyplot(plt) # Display the plot in Streamlit


def plot_product_performance(df):
    """Plots product performance comparisons."""
    df.groupby('Product')['Sales'].sum().plot(kind='bar')
    plt.title('Product Performance Comparison')
    plt.xlabel('Product')
    plt.ylabel('Sales')
    st.pyplot(plt)  # Display the plot in Streamlit

In [ ]:
# --- Streamlit App ---

st.title("AI-Powered Business Intelligence Analysis Tool")

# User input for questions
user_question = st.text_input("Enter your question:")

# Display results and visualizations
if user_question:
    answer = query_data(retriever_chain, user_question)
    st.write("**Answer:**", answer)

    # Trigger visualizations based on user question (example)
    if "sales trend" in user_question.lower():
        plot_sales_trends(sales_df)
    if "product performance" in user_question.lower():
        plot_product_performance(sales_df)


In [ ]:
!streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.138.232.33:8501



In [ ]:
!pip install pyngrok --quiet
from pyngrok import ngrok

# Expose port 8501 (default for Streamlit)
public_url = ngrok.connect(8501).public_url

# Display the public URL in the Streamlit app
st.write(f"[Click here to access the Streamlit app]({public_url})")